In [1]:
#1 Importing Relevant Libraries

import json
import string
import random 
import colorama
colorama.init()
from colorama import Fore, Style, Back

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer 
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

2024-04-04 05:43:18.334259: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 05:43:18.386416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 05:43:18.386482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 05:43:18.387725: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-04 05:43:18.395040: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 05:43:18.396197: I tensorflow/core/platform/cpu_feature_guard.cc:1

True

In [2]:
#2 Loading the Dataset: intents.json

data_file = open('datasets/intents.json').read()
data = json.loads(data_file)

data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Yo', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there', 'Hi there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Bao, your bot assistant specialized in the field of agriculture',
    "I'm Bao, an Artificial Intelligent bot specialized in the field of agriculture"]},
  {'tag': 'name',
   'patterns': ['what is your name ?',
    'what should I call you',
    'whats your name?',
    'How can I call you ?'],
   'responses': ['You can call me Bao, a bot speciali

In [3]:
#3 Extracting data_X(features) and data_Y(Target)

words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow  model/ vocabulary for tags
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X 
# Iterating over all the intents

for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern) # tokenize each pattern 
        words.extend(tokens) #and append tokens to words
        data_X.append(pattern) #appending pattern to data_X
        data_y.append(intent["tag"]) ,# appending the associated tag to each pattern 
    
    # adding the tag to the classes if it's not there already 
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

# initializing lemmatizer to get stem of words
lemmatizer = WordNetLemmatizer()

# lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
# sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [4]:
# 4 Text to Numbers
training = []
out_empty = [0] * len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
    bow = []
    text = lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    # mark the index of class that the current pattern is associated
    # to
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    # add the one hot encoded BoW and associated classes to training 
    training.append([bow, output_row])
# shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y = np.array(list(training[:, 1]))

In [5]:
#5 The Neural Network Model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_Y[0]), activation = "softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=250)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               21504     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 61)                3965      
                                                                 
Total params: 33725 (131.74 KB)
Trainable params: 33725 (131.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/250


2024-04-04 05:43:22.881295: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-04 05:43:22.882256: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


9/9 [==============================] - 1s 3ms/step - loss: 4.1097 - accuracy: 0.0369
Epoch 2/250
9/9 [==============================] - 0s 3ms/step - loss: 3.8539 - accuracy: 0.0775
Epoch 3/250
9/9 [==============================] - 0s 2ms/step - loss: 3.5634 - accuracy: 0.1181
Epoch 4/250
9/9 [==============================] - 0s 2ms/step - loss: 3.2509 - accuracy: 0.1624
Epoch 5/250
9/9 [==============================] - 0s 2ms/step - loss: 2.9336 - accuracy: 0.2435
Epoch 6/250
9/9 [==============================] - 0s 2ms/step - loss: 2.5499 - accuracy: 0.3173
Epoch 7/250
9/9 [==============================] - 0s 2ms/step - loss: 2.2242 - accuracy: 0.3542
Epoch 8/250
9/9 [==============================] - 0s 2ms/step - loss: 2.1615 - accuracy: 0.3838
Epoch 9/250
9/9 [==============================] - 0s 3ms/step - loss: 1.7630 - accuracy: 0.4760
Epoch 10/250
9/9 [==============================] - 0s 2ms/step - loss: 1.6408 - accuracy: 0.5018
Epoch 11/250
9/9 [=======================

In [6]:
#7 to save the trained model
model.save("chat_model")

INFO:tensorflow:Assets written to: chat_model/assets


INFO:tensorflow:Assets written to: chat_model/assets


In [7]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model


input_shape=Input(shape=(len(train_X[0]),))

#Defining intermediate layers
x = Dense(128, activation='relu')(input_shape)
x = Dense(64, activation='relu')(x)


#Defining output layer
outputs = Dense(61, activation='softmax')(x)

#5 The Neural Network Model
model = Model()
#Creating the model
model = Model(inputs=input_shape, outputs=outputs)

#Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

print(model.summary())
model.fit(train_X, train_Y, epochs=250, verbose=1)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 167)]             0         
                                                                 
 dense_3 (Dense)             (None, 128)               21504     
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 61)                3965      
                                                                 
Total params: 33725 (131.74 KB)
Trainable params: 33725 (131.74 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Epoch 1/250
9/9 [==============================] - 1s 3ms/step - loss: 0.6797 - accuracy: 0.0111
Epoch 2/250
9/9 [==============================] - 

In [8]:
#6 Preprocessing the Input

def clean_text(text): 
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab): 
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens: 
    for idx, word in enumerate(vocab):
      if word == w: 
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels): 
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]]) #Contains labels(tags) for highest probability 
  return return_list

def get_response(intents_list, intents_json): 
  if len(intents_list) == 0:
    result = "Sorry! I don't understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents: 
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

In [9]:
#7 to save the trained model
model.save("chat_model1")


INFO:tensorflow:Assets written to: chat_model1/assets


INFO:tensorflow:Assets written to: chat_model1/assets


In [10]:
def chat():
   
    
    while True:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit":
            break

        intents = pred_class(inp, words, classes)
        result = get_response(intents, data)
        print(Fore.GREEN + "Bao:" + Style.RESET_ALL , result)

        # print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)
chat()

Start messaging with the bot (type quit to stop)!
User: 